In [ ]:
import os 
import cv2
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from sklearn.model_selection import train_test_split

# Step 1: Read the text file
data = {}
with open("C:/Users/bibdh/All group steel axels output.txt", 'r') as file:
    lines = file.readlines()
    for line in lines:
        if line.startswith('Image:'):
            image_name = line.split(': ')[1].strip()
        elif line.startswith('Ductility Percentage:'):
            ductility = float(line.split(': ')[1].strip().strip('%'))
        elif line.startswith('Brittleness Percentage:'):
            brittleness = float(line.split(': ')[1].strip().strip('%'))
            data[image_name] = {'ductility': ductility, 'brittleness': brittleness}

# Step 2: Load the images and preprocess
image_folder ="C:/Users/bibdh/All groups steel axles"
images = []
labels = []
for image_name, info in data.items():
    image_path = os.path.join(image_folder, image_name)
    image = cv2.imread(image_path)
    # Perform preprocessing if needed
    # e.g., resize, normalize, convert to grayscale, etc.
    image = cv2.resize(image, (224, 224))  # Resize the image
    image = image / 255.0  # Normalize pixel values
    images.append(image)
    labels.append((info['ductility'], info['brittleness']))

# Step 3: Prepare data for CNN
X = np.array(images)
y = np.array(labels)

# Step 4: Split the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Step 5: Define and train the CNN model
model1 = Sequential()
model1.add(Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)))
model1.add(MaxPooling2D((2, 2)))
model1.add(Conv2D(64, (3, 3), activation='relu'))
model1.add(MaxPooling2D((2, 2)))
model1.add(Flatten())
model1.add(Dense(128, activation='relu'))
model1.add(Dense(2))  # One output for ductility and one for brittleness

model1.compile(optimizer='adam', loss='mse', metrics=['mae'])

epochs = 50
batch_size = 32
model1.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, validation_data=(X_val, y_val))

# Step 6: Evaluate the model
loss, mae = model1.evaluate(X_val, y_val)
print("Validation Loss:", loss)
print("Validation Mean Absolute Error:", mae)

In [ ]:
# Step 7: Make predictions on new images 
def load_and_preprocess_image(image_path):
    image = cv2.imread(image_path)
    image = cv2.resize(image, (224, 224))
    image = image / 255.0
    image = np.expand_dims(image, axis=0)
    return image

new_image_path = "C:\Users\bibdh\29981-V11_Part-1_HV_All levels tempering.jpeg"
new_image = load_and_preprocess_image(new_image_path)
    
# Convert image to grayscale
#gray = cv2.cvtColor(new_image, cv2.COLOR_BGR2GRAY)
# Make predictions
predictions = model1.predict(new_image)
ductility_prediction = predictions[0][0]
brittleness_prediction = 100-ductility_prediction


# Print the predictions
print(f'Predicted Ductility: {ductility_prediction:.2f}%')
print(f'Predicted Brittleness: {brittleness_prediction:.2f}%')